# Paladin analysis

Based on the [simultaneous analysis](simultaneous_analysis.ipynb) we see the frame levels as measured by Paladin are different to each other. Let's measure these values from the images so we can be sure of the method.

## Plan

* Get the list of images for one of the nights in question (20150609)
    * cameras 802 and 806
    * field NG2000
* For each image, measure:
    * raw frame mean
    * raw frame median
    * raw frame variance
    * left overscan
    * right overscan
    * bias subtracted frame mean
    * bias subtracted frame median
    * bias subtracted frame variance

In [82]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import glob
import sqlalchemy as sa
from contextlib import contextmanager
import bz2
from astropy.io import fits

engine = sa.create_engine('mysql+pymysql://sw@ngtsdb/ngts_ops')
write_engine = sa.create_engine('sqlite:///results.db')

In [83]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base(bind=write_engine)
Session = sessionmaker(bind=write_engine)

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

class ResultsRow(Base):
    __tablename__ = 'results'
    
    id = sa.Column(sa.Integer, primary_key=True)
    image_id = sa.Column(sa.BigInteger, nullable=False)
    mjd = sa.Column(sa.Float, nullable=False)
    camera_id = sa.Column(sa.Integer, nullable=False)
    raw_frame_mean = sa.Column(sa.Float, nullable=False)
    raw_frame_median = sa.Column(sa.Float, nullable=False)
    raw_frame_variance = sa.Column(sa.Float, nullable=False)
    left_overscan = sa.Column(sa.Float, nullable=False)
    right_overscan = sa.Column(sa.Float, nullable=False)
    bias_subtracted_frame_mean = sa.Column(sa.Float, nullable=False)
    bias_subtracted_frame_median = sa.Column(sa.Float, nullable=False)
    bias_subtracted_frame_variance = sa.Column(sa.Float, nullable=False)
    
    def __repr__(self):
        return '<Result {}>'.format(self.image_id)
    
Base.metadata.drop_all()
Base.metadata.create_all()

In [53]:
def get_action(camera_id, field):
    r = engine.execute('''
    select distinct action_id from raw_image_list
    where camera_id = %s
    and field = %s
    and night = "20150609"
    ''', (camera_id, field))
    return r.first()[0]

In [54]:
action_802 = get_action(802, "NG2000-4500")
action_806 = get_action(806, "NG2000-4501")
print('Action 802:', action_802, ', action 806:', action_806)

Action 802: 104358 , action 806: 104376


In [56]:
@contextmanager
def open_file(fname):
    if '.bz2' in fname:
        with bz2.BZ2File(fname) as uncompressed:
            with fits.open(uncompressed) as infile:
                yield infile
    else:
        with fits.open(fname) as infile:
            yield infile

* For each image, measure:
    * raw frame mean
    * raw frame median
    * raw frame variance
    * left overscan
    * right overscan
    * bias subtracted frame mean
    * bias subtracted frame median
    * bias subtracted frame variance

In [73]:
def extract_from_frame(fname):
    with open_file(fname) as infile:
        all_data = infile[0].data
        header = infile[0].header
        
    row = ResultsRow(
        image_id=int(header['image_id']),
        mjd=float(header['mjd']),
        camera_id=int(header['cameraid']),
        raw_frame_mean=all_data.mean(),
        raw_frame_median=np.median(all_data),
        raw_frame_variance=all_data.var(),
        left_overscan=all_data[4:, :20].mean(),
        right_overscan=all_data[4:, -15:].mean())
    
    bias_subtracted = all_data - row.right_overscan
    
    row.bias_subtracted_frame_mean = bias_subtracted.mean()
    row.bias_subtracted_frame_median = np.median(bias_subtracted)
    row.bias_subtracted_frame_variance = bias_subtracted.var()
    
    return row   

In [91]:
files_802 = glob.glob('/ngts/testdata/paranal/action{}_observeField/IMAGE*.fits.bz2'.format(action_802))
files_806 = glob.glob('/ngts/testdata/paranal/action{}_observeField/IMAGE*.fits.bz2'.format(action_806))

In [ ]:
session = Session()
session.query(ResultsRow).delete()

for files_list in [files_802, files_806]:
    for fname in files_list:
        row = extract_from_frame(fname)
        session.add(row)
        session.commit()

In [ ]:
a = 10